In [2]:
import pandas as pd

In [3]:
train_df = pd.read_csv('./data/train_data.csv')
test_df = pd.read_csv('./data/test_data.csv')

In [4]:
train_df.head()

,Unnamed: 0,tile_id,datetime,satellite_platform,asset,file_path
0,0,2587,NaN,NaN,documentation,/Users/maxlanger/neuefische/Radiant-Earth-Spot...
1,1,2587,NaN,NaN,field_ids,/Users/maxlanger/neuefische/Radiant-Earth-Spot...
2,2,2587,NaN,NaN,field_info_train,/Users/maxlanger/neuefische/Radiant-Earth-Spot...
3,3,2587,NaN,NaN,labels,/Users/maxlanger/neuefische/Radiant-Earth-Spot...
4,4,2587,NaN,NaN,raster_values,/Users/maxlanger/neuefische/Radiant-Earth-Spot...
